In [18]:
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import joblib

In [19]:
# Load embedder
embedder = FaceNet()
l2_normalizer = Normalizer('l2')

# Data loading
X, y = [], []

In [20]:
for root, dirs, files in os.walk("known_faces"):
    for file in files:
        if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue  # skip non-image files

        img_path = os.path.join(root, file)
        img = cv2.imread(img_path)
        if img is None:
            print(f"[WARNING] Skipping {img_path}, could not read as image.")
            continue

        img = cv2.resize(img, (160, 160))
        embedding = embedder.embeddings([img])[0]
        embedding = l2_normalizer.transform([embedding])[0]
        X.append(embedding)

        # Use the folder name as the label
        label = os.path.basename(root)
        y.append(label)

print(y)        

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━

In [21]:
from sklearn.svm import SVC

X = np.array(X)
y = np.array(y)

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [22]:
# # Train SVM
# model = SVC(kernel='rbf', probability=True)
# model.fit(X, y_encoded)

In [23]:
from sklearn.ensemble import RandomForestClassifier
# Train RandomForest
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X, y_encoded)

RandomForestClassifier(n_estimators=200, random_state=42)

In [24]:
# Save model and label encoder
joblib.dump(model, 'model.pkl')
joblib.dump(encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [25]:
#TESTING

from mtcnn.mtcnn import MTCNN

model = joblib.load('model.pkl')
encoder = joblib.load('label_encoder.pkl')
detector = MTCNN()

cap = cv2.VideoCapture(0)
print("[INFO] Starting webcam... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = detector.detect_faces(frame)
    for result in results:
        x, y, w, h = result['box']
        x, y = max(0, x), max(0, y)
        face = frame[y:y+h, x:x+w]

        if face.size == 0 or face.shape[0] < 10 or face.shape[1] < 10:
            continue

        face_resized = cv2.resize(face, (160, 160))
        embedding = embedder.embeddings([face_resized])[0]
        embedding = l2_normalizer.transform([embedding])[0]

        pred_prob = model.predict_proba([embedding])[0]
        pred_index = np.argmax(pred_prob)
        confidence = pred_prob[pred_index]
        name = encoder.inverse_transform([pred_index])[0]

        label = f'{name} ({confidence:.2f})' if confidence > 0 else 'Unknown'

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[INFO] Starting webcam... Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
